In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set up WebDriver
driver = webdriver.Chrome()

# Open the target URL
url = 'https://www.moneycontrol.com/ipo/listed-ipos'
driver.get(url)

# Maximize the window to reduce element overlap
driver.maximize_window()

# Wait for the page to load
wait = WebDriverWait(driver, 10)

# Function to handle "Load More" button click
def click_load_more():
    try:
        # Locate the "Load More" button
        load_more_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'showmore_showMoreBtn__uaC0V')))
        
        # Scroll to the button and click it
        driver.execute_script("arguments[0].scrollIntoView(true);", load_more_button)
        time.sleep(1)  # Allow for smooth scrolling
        load_more_button.click()
        time.sleep(2)  # Wait for the new data to load
    except ElementClickInterceptedException:
        print("Popup detected. Waiting for it to disappear...")
        time.sleep(6)  # Wait for the popup to close
        return click_load_more()  # Retry clicking the button
    except Exception as e:
        print("Error clicking 'Load More':", e)
        return False
    return True

# Limit the number of rows to load
max_rows = 300
rows_loaded = 0

# Main loop to load all table rows
while rows_loaded < max_rows:
    # Handle popup if present
    try:
        popup_close_button = driver.find_element(By.CLASS_NAME, 'popup-close-button')  # Update selector based on popup element
        popup_close_button.click()
        print("Popup closed.")
        time.sleep(1)  # Short wait after closing popup
    except Exception:
        print("No popup detected.")

    # Try to click "Load More"
    if not click_load_more():
        break

    # Check current rows loaded
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'lxml')
    table = soup.find('table', {'class': 'ClosedIpoTableSec_web_commonTable__T0VXf'})
    if table:
        rows_loaded = len(table.find_all('tr')) - 1  # Exclude header row
        print(f"Rows loaded: {rows_loaded}")

# Get the final page source
html_content = driver.page_source
driver.quit()

# Parse the fully loaded page with BeautifulSoup
soup = BeautifulSoup(html_content, 'lxml')

# Find the table
table = soup.find('table', {'class': 'ClosedIpoTableSec_web_commonTable__T0VXf'})

# Extract headers
headers = [header.get_text(strip=True) for header in table.find('thead').find_all('th')]

# Extract data from rows
data = []
for row in table.find_all('tr')[1:max_rows + 1]:  # Limit rows to max_rows
    cells = row.find_all('td')
    if cells:
        row_data = [cell.get_text(strip=True) for cell in cells]
        data.append(row_data)

# Convert to DataFrame
df = pd.DataFrame(data, columns=headers)


No popup detected.
Popup detected. Waiting for it to disappear...
Rows loaded: 40
No popup detected.
Popup detected. Waiting for it to disappear...
Rows loaded: 60
No popup detected.
Rows loaded: 80
No popup detected.
Rows loaded: 100
No popup detected.
Rows loaded: 120
No popup detected.
Popup detected. Waiting for it to disappear...
Rows loaded: 140
No popup detected.
Popup detected. Waiting for it to disappear...
Popup detected. Waiting for it to disappear...
Rows loaded: 160
No popup detected.
Rows loaded: 180
No popup detected.
Popup detected. Waiting for it to disappear...
Rows loaded: 200
No popup detected.
Popup detected. Waiting for it to disappear...
Popup detected. Waiting for it to disappear...
Popup detected. Waiting for it to disappear...
Popup detected. Waiting for it to disappear...
Rows loaded: 220
No popup detected.
Popup detected. Waiting for it to disappear...
Rows loaded: 240
No popup detected.
Rows loaded: 260
No popup detected.
Rows loaded: 260
No popup detected.

In [4]:
df.head()

,Company Name,,ListingDate,Issue Price,Total Subs,Listing Open (₹),Listing Close (₹),Listing Gain,LTP(₹),As ofTodays Gain,Issue Size
0,DAM Capital Advisors IPO,Mainline,27 Dec 24,₹ 283,82.08x,393,415.1,38.87%,₹ 415.10,5.62%,840.25 Cr
1,Newmalayalam Steel IPO,SME,27 Dec 24,₹ 90,48.19x,90,85.5,0.00%,₹ 85.50,-5.00%,41.73 Cr
2,Concord Enviro Systems IPO,Mainline,27 Dec 24,₹ 701,10.67x,826,827.1,17.83%,₹ 827.10,0.13%,500.33 Cr
3,Sanathan Textiles IPO,Mainline,27 Dec 24,₹ 321,35.12x,422.3,388.35,31.56%,₹ 388.35,-8.04%,550.00 Cr
4,Transrail Lighting IPO,Mainline,27 Dec 24,₹ 432,80.8x,590,553.55,36.57%,₹ 553.55,-6.18%,838.91 Cr


In [5]:
df['Issue Price'] =  pd.to_numeric(df['Issue Price'].str.replace('₹', ''), errors='coerce')
df['Total Subs'] = pd.to_numeric(df['Total Subs'].str.replace('x', ''), errors='coerce')
df['Listing Open (₹)'] = pd.to_numeric(df['Listing Open (₹)'], errors='coerce')
df['Listing Close (₹)'] = pd.to_numeric(df['Listing Close (₹)'], errors='coerce')
df['Listing Gain'] = pd.to_numeric(df['Listing Gain'].str.replace('%', ''), errors='coerce')
df['As_of_Todays_Gain'] = pd.to_numeric(df['As ofTodays Gain'].str.replace('%', ''), errors='coerce')
df['Issue Size'] = pd.to_numeric(df['Issue Size'].str.replace(' Cr', ''), errors='coerce')
df['LTP(₹)'] = pd.to_numeric(df['LTP(₹)'].str.replace('₹', ''), errors='coerce')

In [6]:
df.dtypes

Company Name          object
                      object
ListingDate           object
Issue Price            int64
Total Subs           float64
Listing Open (₹)     float64
Listing Close (₹)    float64
Listing Gain         float64
LTP(₹)               float64
As ofTodays Gain      object
Issue Size           float64
As_of_Todays_Gain    float64
dtype: object

In [7]:
numeric_columns = df.select_dtypes(include=['number'])

#Drop rows with NaN values
numeric_columns.dropna(inplace=True)

# Compute the correlation matrix
correlation_matrix = numeric_columns.corr()

correlation_matrix

,Issue Price,Total Subs,Listing Open (₹),Listing Close (₹),Listing Gain,LTP(₹),Issue Size,As_of_Todays_Gain
Issue Price,1.000000,-0.167316,0.880635,0.868518,-0.154335,0.716062,0.639667,0.018421
Total Subs,-0.167316,1.000000,0.045359,0.061559,0.510503,0.028775,-0.251465,-0.023295
Listing Open (₹),0.880635,0.045359,1.000000,0.994346,0.261708,0.837001,0.507438,0.036525
Listing Close (₹),0.868518,0.061559,0.994346,1.000000,0.268028,0.838224,0.503618,0.045147
Listing Gain,-0.154335,0.510503,0.261708,0.268028,1.000000,0.213403,-0.176352,-0.034896
LTP(₹),0.716062,0.028775,0.837001,0.838224,0.213403,1.000000,0.342278,0.449269
Issue Size,0.639667,-0.251465,0.507438,0.503618,-0.176352,0.342278,1.000000,-0.057385
As_of_Todays_Gain,0.018421,-0.023295,0.036525,0.045147,-0.034896,0.449269,-0.057385,1.000000


In [8]:
# 1. Price Volatility
df['Price Volatility (%)'] = ((df['Listing Close (₹)'] - df['Listing Open (₹)']) / df['Listing Open (₹)']) * 100

# 2. Subscription-to-Issue Ratio
df['Subs to Issue Ratio'] = df['Total Subs'] / df['Issue Size']

# 3. Premium-to-Issue Price Ratio
df['Premium Ratio (%)'] = ((df['LTP(₹)'] - df['Issue Price']) / df['Issue Price']) * 100

# 4. First-Day Performance
df['First-Day Gain (%)'] = ((df['Listing Close (₹)'] - df['Listing Open (₹)']) / df['Issue Price']) * 100

# 5. Risk Score (Example: Composite of key indicators)
df['Risk Score'] = (df['Price Volatility (%)'] + df['Total Subs'] - df['Issue Size']) / 3  # Example formula


In [9]:
df

,Company Name,,ListingDate,Issue Price,Total Subs,Listing Open (₹),Listing Close (₹),Listing Gain,LTP(₹),As ofTodays Gain,Issue Size,As_of_Todays_Gain,Price Volatility (%),Subs to Issue Ratio,Premium Ratio (%),First-Day Gain (%),Risk Score
0,DAM Capital Advisors IPO,Mainline,27 Dec 24,283,82.08,393.0,415.10,38.87,415.10,5.62%,840.25,5.62,5.623410,0.097685,46.678445,7.809187,-250.848863
1,Newmalayalam Steel IPO,SME,27 Dec 24,90,48.19,90.0,85.50,0.00,85.50,-5.00%,41.73,-5.00,-5.000000,1.154805,-5.000000,-5.000000,0.486667
2,Concord Enviro Systems IPO,Mainline,27 Dec 24,701,10.67,826.0,827.10,17.83,827.10,0.13%,500.33,0.13,0.133172,0.021326,17.988588,0.156919,-163.175609
3,Sanathan Textiles IPO,Mainline,27 Dec 24,321,35.12,422.3,388.35,31.56,388.35,-8.04%,550.00,-8.04,-8.039309,0.063855,20.981308,-10.576324,-174.306436
4,Transrail Lighting IPO,Mainline,27 Dec 24,432,80.80,590.0,553.55,36.57,553.55,-6.18%,838.91,-6.18,-6.177966,0.096315,28.136574,-8.437500,-254.762655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,Vibhor Steel Tubes IPO,Mainline,20 Feb 24,151,298.86,425.0,446.25,181.46,212.81,-49.93%,72.17,-49.93,5.000000,4.141056,40.933775,14.072848,77.230000
296,Wise Travel India IPO,SME,19 Feb 24,147,148.45,238.8,221.30,62.45,165.95,-30.51%,94.68,-30.51,-7.328308,1.567913,12.891156,-11.904762,15.480564
297,Entero Healthcare Solutions IPO,Mainline,16 Feb 24,1258,1.53,1228.7,1149.55,-2.33,1426.55,16.10%,NaN,16.10,-6.441768,NaN,13.398251,-6.291733,NaN
298,Polysil Irrigation Systems IPO,SME,16 Feb 24,54,6.57,45.5,43.00,-15.74,26.80,-41.10%,17.43,-41.10,-5.494505,0.376936,-50.370370,-4.629630,-5.451502


https://www.mckinsey.com/capabilities/strategy-and-corporate-finance/our-insights/a-new-way-to-measure-ipo-success
https://www.indiainfoline.com/knowledge-center/ipo/ipo-valuation
https://www.pwc.com/us/en/services/consulting/deals/library/successful-ipo.html
https://www.chittorgarh.com/book-chapter/ipo-kpis-key-performance-indicators-explain/39/